In [1]:
# !pip install selenium 
# !pip install beautifulsoup4
#
#!pip install webdriver-manager
#!pip install lxml
#!pip install spacy
#!python3 -m spacy download en_core_web_lg
#!python3 -m pip install "pymongo[srv]"

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from datetime import datetime
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import time
import spacy
nlp = spacy.load('en_core_web_lg')
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

# Access the MongoDB URI
passwordL = os.getenv('LINKEDIN_PASSWORD')


# Creating a webdriver instance
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# This instance will be used to log into LinkedIn

# Opening linkedIn's login page
driver.get("https://linkedin.com/uas/login")

# waiting for the page to load
time.sleep(5)

# entering username
username = driver.find_element(By.ID, "username")

# In case of an error, try changing the element
# tag used here.

# Enter Your Email Address
username.send_keys("rohit108180@gmail.com") 

# entering password
pword = driver.find_element(By.ID, "password")
# In case of an error, try changing the element 
# tag used here.

# Enter Your Password
pword.send_keys(passwordL)	 

# Clicking on the log in button
# Format (syntax) of writing XPath --> 
# //tagname[@attribute='value']
driver.find_element(By.XPATH, "//button[@type='submit']").click()
# In case of an error, try changing the
# XPath used here.

# profile_url = "https://www.linkedin.com/feed"
 
# driver.get(profile_url)  






In [3]:
mongo_uri = os.getenv('MONGO_URI')
# SETUP THE MONGO_CONNECTION
uri = mongo_uri if mongo_uri else "mongodb://localhost:27017"


# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)


Pinged your deployment. You successfully connected to MongoDB!


In [4]:
start = time.time()

# will be used in the while loop
initialScroll = 0
finalScroll = 1000

while True:
	driver.execute_script(f"window.scrollTo({initialScroll},{finalScroll})")
	# this command scrolls the window starting from
	# the pixel value stored in the initialScroll 
	# variable to the pixel value stored at the
	# finalScroll variable
	initialScroll = finalScroll
	finalScroll += 1000

	# we will stop the script for 3 seconds so that 
	# the data can load
	time.sleep(3)
	# You can change it as per your needs and internet speed

	end = time.time()

	# We will scroll for 20 seconds.
	# You can change it as per your needs and internet speed
	if round(end - start) > 400:
		break


In [5]:

src = driver.page_source
 
# Now using beautiful soup
soup = BeautifulSoup(src, 'lxml')
# f = open("feed.txt", "w")
# f.write(src)
# f.close()

In [6]:
# fie-impression-container
feeds = soup.find_all('div', {'id': 'fie-impression-container'})

In [7]:
print(len(feeds))

251


In [8]:
posts = [];
# getting the names and info about of POSTER
for feed in feeds:
    poster = feed.find('span', {'class': 'update-components-actor__name'})
    if poster :
        poster = poster.find('span', {'class': 'visually-hidden'}).get_text()
        
    
    poster_bio = feed.find('span', {'class': 'update-components-actor__description'})
    if poster_bio :
        poster_bio = poster_bio.find('span', {'class': 'visually-hidden'}).get_text()
    if poster == "LinkedIn for Marketing" or poster == "poster Indian Startup News" or poster_bio == "Promoted" or not poster or not poster_bio:
        continue
    profile = feed.find('a', {'class': 'update-components-actor__sub-description-link'})
    profile = profile["href"] if profile else profile;
    feed_text = feed.find('span', {'class': 'break-words tvm-parent-container'});

    hrefs = feed_text.find_all("a") if feed_text else []
    href_map = {}
    for href in hrefs:
        link_text = href.get_text() if href else ""
        if link_text.find("hashtag#") != -1 or link_text == "" :
            continue;
        href_map[href.get_text()] = href["href"] if href else href;
    post = {}
    post["poster"] = poster;
    post["poster_bio"] =  poster_bio;
    post["profile"] =  profile;
    post["feed_text"] =  feed_text.get_text() if feed_text else feed_text
    post["href_map"] = href_map;
    post["createdAt"] = datetime.utcnow()
    post["status"] = "pending"
    posts.append(post)
    print(post)
    print("\n")
    
    
    

{'poster': 'Saurabh Gupta', 'poster_bio': 'Executive Director at J.P. Morgan, India Lead Asset Management Client Service', 'profile': 'https://www.linkedin.com/in/saurabh-gupta-8bbbb523?miniProfileUrn=urn%3Ali%3Afsd_profile%3AACoAAAT7lc8BfBOEFEOFFsSL-nDi3qV-D3dgWIE', 'feed_text': '\n Hi All, we are looking for an Associate in our Client Account Management team based out of Bengaluru. The role will involve servicing our Institutional Client requests, coordinating with various internal JPM teams and delivering superior Client Experience (CX). If you are keen and passionate about Client Servicing, thrive in challenging but exciting situations and can multi-task then please apply on the link below.https://lnkd.in/dGdirgAt\n', 'href_map': {'https://lnkd.in/dGdirgAt': 'https://lnkd.in/dGdirgAt'}, 'createdAt': datetime.datetime(2024, 8, 21, 15, 12, 35, 257063), 'status': 'pending'}


{'poster': 'Palak Awasthi', 'poster_bio': 'Software Engineer 2 @PayPal | 70k+ @linkedIn| Women Techmakers Amba

In [9]:

# # Extracting the HTML of the complete introduction box
# # that contains the name, company name, and the location
# feeds = soup.find_all('div', {'class': 'feed-shared-update-v2__description-wrapper'})
 

In [10]:
    
# feed_texts = [ feed.find('span', {'class': 'break-words tvm-parent-container'}) for feed in feeds]
# for feed_text in feed_texts:
#     print(feed_text.get_text())
#     print(feed_text.get_text().find("hiring"))
#     print("\n")

In [11]:

# Given text
# given_text = nlp("🚀 We're Hiring! Join Our Dynamic Team! We're excited to announce that [Company Name] is expanding, and we're looking for passionate individuals to join us. If you're someone who thrives in a fast-paced environment and loves solving challenging problems, we want to hear from you! Open Positions:Software EngineerLocation: [City, Country]Apply HereData ScientistLocation: [City, Country]Apply HereProduct ManagerLocation: [City, Country]Apply HereWhy [Company Name]?Competitive salary and benefitsOpportunity to work with a talented teamFlexible working hours and remote work optionsProfessional development and growth opportunitiesInterested?Click on the links above to apply or visit our Careers Page. Share this post with anyone who might be a great fit! Let's build something amazing together.#Hiring #JobOpportunity #JoinOurTeam #TechJobs")
# given_text2=nlp("I am a passionate Computer Engineering student with a minor in Artificial Intelligence, currently seeking full-time Software Engineering roles. I have experience in developing and optimizing software solutions, with strong skills in C/C++, SQL, JavaScript, ReactJs, Node.js, and database management (SQL, NoSQL). I am particularly interested in positions that involve: Backend development using Node.js and SQL/NoSQL databases. API development and optimization for scalable applications. Collaborative projects in agile environments. Opportunities for growth in machine learning and AI integration. I have a proven track record of success, including leading teams in hackathons and internships where I contributed to significant performance improvements. I am looking for a role where I can leverage my technical skills and leadership abilities to contribute to innovative projects. If your company is hiring for roles that match these interests, I would love to connect! #SoftwareEngineering #BackendDevelopment #NodeJS #AI #APIDevelopment #TechJobs #Hiring");
# # Filter texts based on similarity
# threshold = 0.7  # Set your threshold

# filtered_texts = [post for post in posts if post["feed_text"] and given_text.similarity(nlp(post["feed_text"])) > threshold]


# for feed_text in filtered_texts:
#     print(feed_text)
#     print("\n")

In [12]:
# SAVE THE DATA TO THE MONGO DB



In [13]:
db = client["posts"]
posts_collection = db["posts"]


In [14]:
posts_collection.insert_many(posts)

InsertManyResult([ObjectId('66c603ec61d58c42c3930aad'), ObjectId('66c603ec61d58c42c3930aae'), ObjectId('66c603ec61d58c42c3930aaf'), ObjectId('66c603ec61d58c42c3930ab0'), ObjectId('66c603ec61d58c42c3930ab1'), ObjectId('66c603ec61d58c42c3930ab2'), ObjectId('66c603ec61d58c42c3930ab3'), ObjectId('66c603ec61d58c42c3930ab4'), ObjectId('66c603ec61d58c42c3930ab5'), ObjectId('66c603ec61d58c42c3930ab6'), ObjectId('66c603ec61d58c42c3930ab7'), ObjectId('66c603ec61d58c42c3930ab8'), ObjectId('66c603ec61d58c42c3930ab9'), ObjectId('66c603ec61d58c42c3930aba'), ObjectId('66c603ec61d58c42c3930abb'), ObjectId('66c603ec61d58c42c3930abc'), ObjectId('66c603ec61d58c42c3930abd'), ObjectId('66c603ec61d58c42c3930abe'), ObjectId('66c603ec61d58c42c3930abf'), ObjectId('66c603ec61d58c42c3930ac0'), ObjectId('66c603ec61d58c42c3930ac1'), ObjectId('66c603ec61d58c42c3930ac2'), ObjectId('66c603ec61d58c42c3930ac3'), ObjectId('66c603ec61d58c42c3930ac4'), ObjectId('66c603ec61d58c42c3930ac5'), ObjectId('66c603ec61d58c42c3930a

In [15]:
print(len(posts))
    

227
